In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from pathlib import Path
from tqdm import tqdm
import holoviews as hv
import hvplot.pandas
import hdf5storage

In [ ]:
hv.extension('bokeh')

In [ ]:
#records_file_path = "/kucresis/scratch/tteisberg_sta/scripts/opr_user_tmp/headers/rds/2018_Antarctica_BaslerJKB/PEL/JKB2u/Y20b/RADnh5/bxds.mat"
#records_file_path = "/resfs/GROUPS/CRESIS/dataproducts/opr_support/records/rds/2018_Antarctica_BaslerJKB/records_20190114_98.mat"
#records_file_path = "/resfs/GROUPS/CRESIS/dataproducts/opr_support/records/rds/2019_Greenland_P3/records_20190328_05.mat"
records_file_path = "/resfs/GROUPS/CRESIS/dataproducts/opr_support/records/rds/2017_Antarctica_BaslerJKB/records_20171229_02.mat"

records = hdf5storage.loadmat(records_file_path)
records.keys()

In [ ]:
import utig_radar_loading.time_util as time_util

time_util.parse_timestamp_with_epoch(records['gps_time'][0], 2019)

In [ ]:
plt.plot(records['gps_time'].squeeze())

In [ ]:
records

In [ ]:
def strip_singleton_arrays(a):
    if isinstance(a, np.ndarray) and len(a) == 1:
        return strip_singleton_arrays(a[0])
    elif isinstance(a, np.ndarray):
        return np.array([strip_singleton_arrays(x) for x in a])
    else:
        return a

filenames = strip_singleton_arrays(records['relative_filename'])
relative_rec_num = strip_singleton_arrays(records['relative_rec_num'])
offsets_split = []

# If relative_rec_num is a scalar (not an array), make it an array
if np.isscalar(relative_rec_num):
    relative_rec_num = np.array([relative_rec_num])
    filenames = np.array([filenames])

for idx in range(len(relative_rec_num)):
    start_idx = int(relative_rec_num[idx])-1
    if idx == len(relative_rec_num)-1:
        end_idx = None
    else:
        end_idx = int(relative_rec_num[idx+1])-1
    
    offsets_split.append(records['offset'][:, start_idx:end_idx, :])

for o in offsets_split:
    print(np.shape(o))

In [ ]:
tmp_hdr_dir = '/kucresis/scratch/tteisberg_sta/scripts/opr_user_tmp/headers/rds/2017_Antarctica_BaslerJKB/'

In [ ]:
# fig, ax = plt.subplots()

# offset_idx = 0

# # Find temporary header for the corresponding file
# tmp_hdr_path = Path(tmp_hdr_dir) / Path(*Path(file_path).parts[-5:-1]) / "bxds.mat"
# tmp_hdr = hdf5storage.loadmat(tmp_hdr_path)

# # Get size of file in bytes and add it as a horizontal line on the plot
# file_path = filenames[offset_idx]
# print(file_path)
# if os.path.exists(file_path):
#     file_size = os.path.getsize(file_path)
#     ax.axhline(y=file_size, color='red', linestyle='--', linewidth=2, label=f'File size ({file_size:,} bytes)')

# for ch_idx in range(4):
#     tmp = offsets_split[offset_idx][0, :, ch_idx]
#     tmp[tmp == -2**31] = np.nan
#     ax.plot(tmp, label=f'Ch {ch_idx}')

#     tmp2 = tmp_hdr['offset'][0, :, ch_idx].astype(float)
#     tmp2[tmp2 == -2**31] = np.nan
#     ax.plot(tmp2, label=f'Ch {ch_idx} (tmp_hdr)', linestyle=':')

# ax.set_xlabel('Record Number')
# ax.set_ylabel('Byte Offset')
# ax.legend()
# ax.grid(True, alpha=0.3)

In [ ]:
# Set up data
offset_idx = 0
file_path = filenames[offset_idx]
tmp_hdr_path = Path(tmp_hdr_dir) / Path(*Path(file_path).parts[-5:-1]) / "bxds.mat"
tmp_hdr = hdf5storage.loadmat(tmp_hdr_path)

# Create dataframe for plotting
plot_data = {}
x_vals = np.arange(offsets_split[offset_idx].shape[1])

# Add channel data from records
for ch_idx in range(4):
    tmp = offsets_split[offset_idx][0, :, ch_idx].astype(float)
    tmp[tmp == -2**31] = np.nan
    plot_data[f'Ch {ch_idx}'] = tmp
    
    # Add tmp_hdr data
    tmp2 = tmp_hdr['offset'][0, :, ch_idx].astype(float)
    tmp2[tmp2 == -2**31] = np.nan
    # Pad with NaN if needed to match length
    if len(tmp2) < len(x_vals):
        tmp2 = np.concatenate([tmp2, np.full(len(x_vals) - len(tmp2), np.nan)])
    plot_data[f'Ch {ch_idx} (tmp_hdr)'] = tmp2

# Create dataframe
df = pd.DataFrame(plot_data, index=x_vals)

# Create the plot
plot = df.hvplot.line(
    xlabel='Record Number',
    ylabel='Byte Offset', 
    title=f'Byte Offsets for {os.path.basename(file_path)}',
    width=800,
    height=500,
    legend='top_left'
).opts(show_grid=True)

# Add file size line if file exists
if os.path.exists(file_path):
    file_size = os.path.getsize(file_path)
    print(file_size)
    file_size_line = hv.HLine(file_size).opts(
        color='red',
        line_dash='dashed',
        line_width=2
    )
    plot = plot * file_size_line

plot

In [ ]:
2147483647

In [ ]:
2**32/2

In [ ]:
records['relative_filename']

In [ ]:
records['offset']